# Acesso ao GCS com DuckDB

Você pode acessar o bucket manualmente através do [console do google cloud](https://console.cloud.google.com/storage/browser/fiat_betim).

Ao executar todas as células você automaticamente grava a versão mais recente da tabela pronta pra predição e a tabela completa para exploração dentro do bucket. Posteriomente, pode-se refatorar e automatizar o processo com o Airflow.


### Setup

In [2]:
import duckdb

duckdb.sql('INSTALL httpfs')
duckdb.sql('LOAD httpfs')
duckdb.sql("SET s3_endpoint='storage.googleapis.com'")


### Credenciais

In [3]:
# configuring settings in the Google Console.
duckdb.sql(f"SET s3_access_key_id='GOOG1E2RA7XOROH2K2IOR6BAL6I4IUEELSXABOH2YSEA5JDP5VC44U2MTJP3B'")
# configuring settings in the Google Console.
duckdb.sql(f"SET s3_secret_access_key='Au0oXGxypbfM2yvcGqUSte6I3U0r2h6oF/q38IcG'")

### Lendo todos os arquivos dentro do diretório /raw

In [3]:
create_table_sql = """
    CREATE TABLE fiat_betim AS
    SELECT
      "CNPJ da Carga" AS cnpj_carga,
      "Capacidade da Carga (MW)" AS capacidade_carga_mw,
      "Carga" AS carga,
      "Cidade" AS cidade,
      "Classe do perfil do agente" AS classe_perfil_agente,
      "Consumo de energia ajustado da parcela cativa da carga parcialmente livre - MWh (RC_CAT c,j)" AS consumo_rc_cat,
      "Consumo de energia ajustado de uma parcela de carga - MWh (RC c,j)" AS consumo_rc,
      "Consumo de energia no ponto de conexão da parcela de carga - MWh (MED_C c,j)" AS consumo_med_c,
      "Consumo no Ambiente Livre da parcela de carga - MWh (RC_AL c,j)" AS consumo_rc_al,
      "Cód. Carga" AS cod_carga,
      "Cód. Perfil" AS cod_perfil,
      "Cód. Perfil Distribuidora" AS cod_perfil_distribuidora,
      "Data" AS data,
      "Data de Migração" AS data_migracao,
      "Estado" AS estado,
      "HH" AS hora,
      "Nome Empresarial" AS nome_empresarial,
      "Ramo de Atividade" AS ramo_atividade,
      "Sigla" AS sigla,
      "Sigla Perfil Distribuidora" AS sigla_perfil_distribuidora,
      "Submercado" AS submercado,
      "CNAE" AS cnae
    FROM read_parquet('s3://fiat_betim/raw/*.parquet', union_by_name=true)
"""

duckdb.sql(create_table_sql)

In [7]:
duckdb.sql("describe fiat_betim")

┌────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│        column_name         │ column_type │  null   │   key   │ default │ extra │
│          varchar           │   varchar   │ varchar │ varchar │ varchar │ int32 │
├────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ cnpj_carga                 │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ capacidade_carga_mw        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ carga                      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ classe_perfil_agente       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ consumo_rc_cat             │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ consumo_rc                 │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ consumo_med_c              │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ consumo_rc_al              │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ co

### Dropando variáveis redundantes

- `ramo_atividade` e `cnae` estão vinculados com o `cnpj_carga`
- `cidade` e `estado` são BETIM, MG.
- `submercado` refere-se à região geoelétrica do local determinado pelo SIN, do qual é sudeste para Betim, MG.
- `nome_empresarial` refere-se ao nome que consta no cadastro de `cnpj_carga`, FCA FIAT CHRYSLER AUTOMOVEIS BRASIL LTDA.

In [5]:
drop_columns_query = """
ALTER TABLE fiat_betim DROP COLUMN ramo_atividade;
ALTER TABLE fiat_betim DROP COLUMN cnae;
ALTER TABLE fiat_betim DROP COLUMN cidade;
ALTER TABLE fiat_betim DROP COLUMN estado;
ALTER TABLE fiat_betim DROP COLUMN submercado;
ALTER TABLE fiat_betim DROP COLUMN nome_empresarial;
ALTER TABLE fiat_betim DROP COLUMN cod_perfil_distribuidora;
"""

duckdb.sql(drop_columns_query)

### Corrigindo formatação

In [8]:
format_columns_query="""
UPDATE fiat_betim
SET
    hora = CAST(REPLACE(hora, ',', '.') AS BIGINT),
    capacidade_carga_mw = CAST(REPLACE(capacidade_carga_mw, ',', '.') AS DOUBLE),
    consumo_rc_cat = CAST(REPLACE(consumo_rc_cat, ',', '.') AS DOUBLE),
    consumo_rc = CAST(REPLACE(consumo_rc, ',', '.') AS DOUBLE),
    consumo_med_c = CAST(REPLACE(consumo_med_c, ',', '.') AS DOUBLE),
    consumo_rc_al = CAST(REPLACE(consumo_rc_al, ',', '.') AS DOUBLE),
    cnpj_carga = REPLACE(cnpj_carga, ',', '.'),
    data_migracao = CAST(data_migracao AS DATE);

"""
duckdb.sql(format_columns_query)

### Criando Schema

Isso pode ser útil para o BigQuery.

In [11]:
create_schema_query="""
CREATE TABLE new_fiat_betim (
    cnpj_carga VARCHAR,
    capacidade_carga_mw DOUBLE,
    carga VARCHAR,
    classe_perfil_agente VARCHAR,
    consumo_rc_cat DOUBLE,
    consumo_rc DOUBLE,
    consumo_med_c DOUBLE,
    consumo_rc_al DOUBLE,
    cod_carga BIGINT,
    cod_perfil BIGINT,
    data DATE,
    data_migracao DATE,
    hora BIGINT,
    sigla VARCHAR,
    sigla_perfil_distribuidora VARCHAR
);

"""
duckdb.sql(create_schema_query)

### Criando nova tabela e inserindo tabela anterior

In [12]:
create_schema_query="""
INSERT INTO new_fiat_betim
SELECT
    cnpj_carga,
    CAST(NULLIF(capacidade_carga_mw, '') AS DOUBLE),
    carga,
    classe_perfil_agente,
    CAST(NULLIF(consumo_rc_cat, '') AS DOUBLE),
    CAST(NULLIF(consumo_rc, '') AS DOUBLE),
    CAST(NULLIF(consumo_med_c, '') AS DOUBLE),
    CAST(NULLIF(consumo_rc_al, '') AS DOUBLE),
    cod_carga,
    cod_perfil,
    data,
    CAST(data_migracao AS DATE),
    CAST(hora AS BIGINT),
    sigla,
    sigla_perfil_distribuidora
FROM fiat_betim;
DROP TABLE fiat_betim;
"""
duckdb.sql(create_schema_query)

In [13]:
duckdb.sql("describe new_fiat_betim")

┌────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│        column_name         │ column_type │  null   │   key   │ default │ extra │
│          varchar           │   varchar   │ varchar │ varchar │ varchar │ int32 │
├────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ cnpj_carga                 │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ capacidade_carga_mw        │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ carga                      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ classe_perfil_agente       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ consumo_rc_cat             │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ consumo_rc                 │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ consumo_med_c              │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ consumo_rc_al              │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ co

### Salvando a tabela final no diretório /clean

Esta tabela apenas contém as variáveis de interesse para a predição como o consumo, data e hora. E referente à filial FA FILIAL MEC.

#### Pressupostos

H1: `consumo_med_c` reflete o valor mais próximo do valor medido pela fábrica.
- O consumo será medida pelo consumo de energia no ponto de conexão da parcela de carga - MWh (MED_C c,j), pois é a versão bruta e mais imediata do consumo. É o valor mais próximo do consumo real e imediato da fábrica.

H2: A filial identificada com a sigla `FA FILIAL MEC` é a nossa fábrica de interesse por focar exclusivamente na fabricação.
- Se supõe que a filial se dedica exclusivamente na fabricação de motores para automóveis, camionetas e utilitários.

In [17]:
export_filial_mec_dataset="""
COPY(SELECT
    data,
    hora,
    consumo_med_c,
FROM new_fiat_betim
WHERE sigla = 'FA FILIAL MEC')
TO 's3://fiat_betim/clean/fiat_filial_mec.parquet' (FORMAT PARQUET);
"""
duckdb.sql(export_filial_mec_dataset)

### Salvando a tabela completa para exploração
A tabela contendo todas as colunas para fins de exploração.
A versão completa possibilita

In [ ]:
export_all_dataset="""
COPY(SELECT *
FROM new_fiat_betim)
TO 's3://fiat_betim/clean/fiat_all.parquet' (FORMAT PARQUET);
"""
duckdb.sql(export_all_dataset)